In [61]:
import pandas as pd
data_type = 'month'
date_list = {
    "season": 4,
    "month": 12,
}
for data_name in ['company', 'r1', 'r2', 'r1-region', 'r2-region']:
# for data_name in ['company']:
    if not os.path.exists(f'data/{data_type}_{data_name}/task1/train'):
        os.makedirs(f'data/{data_type}_{data_name}/task1/train')
    if not os.path.exists(f'data/{data_type}_{data_name}/task1/eval'):
        os.makedirs(f'data/{data_type}_{data_name}/task1/eval')
    if not os.path.exists(f'data/{data_type}_{data_name}/task1/test'):
        os.makedirs(f'data/{data_type}_{data_name}/task1/test')
    if not os.path.exists(f'data/{data_type}_{data_name}/task2'):
        os.makedirs(f'data/{data_type}_{data_name}/task2')


    df = pd.read_csv(f'../data/{data_type}/job_count_{data_name}.csv')
    col_index = [f"{i}_id" for i in data_name.split('-')]
    for col in col_index + ['skill_id']:
        df[col] = df[col].apply(int)
    col_num = [df[col].nunique() for col in col_index]
    df[data_name] = df.apply(lambda x: int(sum([x[v]*col_num[i-1] if i > 0 else x[v] for i,v in enumerate(col_index)])),axis=1)
    df['skill_id'] = df['skill_id'] + 200000

    pd.DataFrame(sorted(df.reset_index()[data_name].unique().tolist() + df.reset_index()['skill_id'].unique().tolist())).reset_index().to_csv(f'data/{data_type}_{data_name}/entities.dict', index=None, sep='\t', header=None)
    pd.DataFrame([0]* df.reset_index()['skill_id'].nunique()).to_csv(f'data/{data_type}_{data_name}/skill2cluster.tsv', index=None, header=None)
    pd.DataFrame([100000]).to_csv(f'data/{data_type}_{data_name}/relations.dict', sep='\t', header=None)

    name_df = pd.DataFrame(sorted(df.reset_index()[data_name].unique().tolist() + df.reset_index()['skill_id'].unique().tolist()))
    real_name_list = pd.read_csv(f'../../dataset/name/{data_name}_name.list')[f'{data_name}_name'].tolist()
    skill_name_list = pd.read_csv(f'../../dataset/name/skill_name.list')['skill'].tolist()
    name_list = name_df[0].progress_apply(lambda x: real_name_list[x] if x<200000 else skill_name_list[x-200000])
    name_list.to_csv(f'{data_name}.list', sep='\t', header=None, index=None)
    os.system(f'python bert.py {data_type}_{data_name}')
    

    pre_train_data = [f"{year}-{month:02d}" for year in range(2021, 2023) for month in range(1, date_list[data_type] + 1)]
    pre_df = df[[data_name] + pre_train_data + ['skill_id']]
    pre_df = pre_df.set_index([data_name, 'skill_id'])
    pre_df['demand'] = pre_df.sum(axis=1)
    pre_df = pre_df[['demand']].reset_index()
    pre_df['relation'] = 100000
    pre_df = pre_df[pre_df['demand']>100]
    ran_pre_df = pre_df.sample(n=len(pre_df),axis=0,random_state=42)

    ran_pre_df = ran_pre_df[[data_name, 'relation', 'skill_id', 'demand']]
    pre_train_df = ran_pre_df[:int(0.95 *len(pre_df))]
    pre_eval_df = ran_pre_df[int(0.95 *len(pre_df)):int(0.99 *len(pre_df))]
    pre_test_df = ran_pre_df[int(0.99 *len(pre_df)):]
        
    pre_train_df.to_csv(f'data/{data_type}_{data_name}/task1/train/triplet_value.tsv', index=None, sep='\t', header=None)
    pre_eval_df.to_csv(f'data/{data_type}_{data_name}/task1/eval/triplet_value.tsv', index=None, sep='\t', header=None)
    pre_test_df.to_csv(f'data/{data_type}_{data_name}/task1/test/triplet_value.tsv', index=None, sep='\t', header=None)

    pre_train_df.to_csv(f'data/{data_type}_{data_name}/task2/old_train_triplet_value.tsv', index=None, sep='\t', header=None)
    pre_eval_df.to_csv(f'data/{data_type}_{data_name}/task2/old_eval_triplet_value.tsv', index=None, sep='\t', header=None)
    pre_test_df.to_csv(f'data/{data_type}_{data_name}/task2/old_test_triplet_value.tsv', index=None, sep='\t', header=None)

100%|██████████| 3/3 [00:01<00:00,  2.95it/s]


In [66]:
import pandas as pd
for data_name in ['company', 'r1', 'r2', 'r1-region', 'r2-region']:
    df = pd.read_csv(f'../data/{data_type}/job_count_{data_name}.csv')
    col_index = [f"{i}_id" for i in data_name.split('-')]
    for col in col_index + ['skill_id']:
        df[col] = df[col].apply(int)
    col_num = [df[col].nunique() for col in col_index]
    df[data_name] = df.apply(lambda x: int(sum([x[v]*col_num[i-1] if i > 0 else x[v] for i,v in enumerate(col_index)])),axis=1)
    df = df.drop(col_index, axis=1).set_index([data_name, 'skill_id'])
    for i, col in enumerate(df.columns):
        sub_df = df[[col]].reset_index()
        sub_df['relation'] = 100000
        sub_df = sub_df[[data_name, 'relation', 'skill_id', col]]
        sub_df = sub_df[sub_df[col]>0]
        sub_df['skill_id'] = sub_df['skill_id'] + 200000

        ran_pre_df = sub_df.sample(n=len(sub_df),axis=0,random_state=42).reset_index().drop('index', axis=1)
        
        train_df = ran_pre_df[:int(0.95 *len(ran_pre_df))]
        eval_df = ran_pre_df[int(0.95 *len(ran_pre_df)):]
        if not os.path.exists(f'data/{data_type}_{data_name}/task2/{i}/train'):
            os.makedirs(f'data/{data_type}_{data_name}/task2/{i}/train')
        if not os.path.exists(f'data/{data_type}_{data_name}/task2/{i}/eval'):
            os.makedirs(f'data/{data_type}_{data_name}/task2/{i}/eval')
        train_df.to_csv(f'data/{data_type}_{data_name}/task2/{i}/train/triplet_value.tsv', index=None, sep='\t', header=None)
        eval_df.to_csv(f'data/{data_type}_{data_name}/task2/{i}/eval/triplet_value.tsv', index=None, sep='\t', header=None)
        sub_df.to_csv(f'data/{data_type}_{data_name}/task2/{i}/triplet_value.tsv', index=None, sep='\t', header=None)

In [63]:
data_list = ['company', 'r1', 'r2', 'r1-region', 'r2-region']
cur_root_dir = '/individual/chenxi02/conferences/NeuIPS24/benchmark/baselines/Pre-DyGAE/data'
import pandas as pd
ans = {}
for data_name in data_list:
    ans[data_name] = []
    for i in range(date_list[data_type]*3):
        file_path = f'{cur_root_dir}/{data_name}/task2/{i}/triplet_value.tsv'
        x_i = pd.read_csv(file_path, header=None, sep='\t')
        x_i['index'] = x_i[0].apply(str) + '-' + x_i[2].apply(str)
        ans[data_name].append(x_i)

for data_name in data_list:
    for i in range(date_list[data_type]*3):
        current_i = ans[data_name][i]
        pre_i = ans[data_name][i - 1]
        x_i = pd.merge(current_i, pre_i, how='inner', on='index')
        current_i = current_i.set_index('index')
        x_i['diff'] = x_i['3_y'] - x_i['3_x']
        diff_i = x_i[['index', 'diff']]
        diff_i['diff'] = diff_i['diff'].apply(lambda x: 3 if x > 0 else 2)
        current_i['diff'] = 1
        current_i['diff'][diff_i['index']] = diff_i['diff'].tolist()
        current_i.to_csv(f'{cur_root_dir}/{data_type}_{data_name}/task2/{i}/diff_triplet_value.tsv', header=None, sep='\t', index=None)

/tmp/ipykernel_181664/3365603392.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_i['diff'] = diff_i['diff'].apply(lambda x: 3 if x > 0 else 2)
/tmp/ipykernel_181664/3365603392.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_i['diff'][diff_i['index']] = diff_i['diff'].tolist()
